# 전처리

## (1) Word embedding model import

In [63]:
from gensim.models import Word2Vec
embedding_model = Word2Vec.load(r'C:\Users\thehb\Code\sentenceExtraction_with_Article\embedding_model')

C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [64]:
word_vectors = embedding_model.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]

## (2) Train data import

In [65]:
# csv파일 불러옴
import csv
import re

raw = []
label_damage = []
f = open(r'C:\Users\thehb\Code\sentenceExtraction_with_Article\sentences_damage2.csv', 'r', encoding='ansi')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line[0])
    label_damage.append(line[1])
f.close()  

# 빈 text 제거 & 특수문자 제거
sents = []
for i in range(len(raw)):
    if(raw[i] is not ' '):
        newsent = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw[i])
        sents.append(newsent)

In [66]:
# 품사 태깅(주어진 텍스트를 형태소 단위로 나누고 나눠진 형태소를 해당하는 품사와 함께 리스트화)
# stemming(단어들을 원형으로 포현), normalization(표현 방법이 다른 단어들을 통합시켜 같은 단어로 만듬)
import json
import os
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc): # 형태소 분석기인 Okt를 이용해서 품사를 태깅시킴
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

os.chdir(r'C:\Users\thehb\Code\sentenceExtraction_with_Article') # 경로 설정

## (3) 문장을 vector로 변환

In [67]:
token_sents = [] # word_vectors의 key가 '단어/품사'의 형식으로 되어 있기 때문에, 같은 형식으로 만들기 위해 각 문장별로 tokenize를 적용함.
for sent in sents:
    token_sents.append(tokenize(sent))

In [74]:
vec_sents = []

for token_sent in token_sents:
    temp_sent = []
    for token_word in token_sent:
        # '만조/Noun' 같은 단어가 word_vectors 안에 없는 경우가 있어서
        # 이런 경우에는 단어를 추가하지 않고, 지나가도록 만듦.
        try:
            temp_sent.append(word_vectors.get_vector(token_word))
        except KeyError:# key가 존재하지 않을 때
            pass # 그냥 지나감        
    vec_sents.append(temp_sent)

# 모델링

## (1) 문장 속 단어들의 Vector의 평균값을 이용해서 문장 분류

In [111]:
import numpy as np

sentences = np.asarray(vec_sents)
mean_sents = []
for i in range(len(sentences)):
    mean_sents.append(np.mean(sentences[i], axis=0))

In [138]:
np.sum(np.array(label_damage) == '1')/len(label_damage)

0.42276422764227645

In [120]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression()
scores = cross_val_score(logreg, mean_sents, label_damage, cv=5) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))
# 전체에서 0인 것 57%, 1인 것 43%인데 결과가 84%면 괜찮은 듯.. 

cross-val-score 
[0.82894737 0.83783784 0.80821918 0.89041096 0.84931507]
cross-val-score.mean 
0.843


C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this

## (2) CNN을 이용해서 문장 분류